In [1]:
# CÉLULA 1: Importações e Definição das Funções do Pipeline

import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import matplotlib.pyplot as plt
import seaborn as sns

# --- FUNÇÃO 1: Criar Labels com o Triple-Barrier Method ---
def create_labels(df, look_forward_period, profit_target_pct, stop_loss_pct):
    df['future_price'] = df['close'].shift(-look_forward_period)
    df['price_change'] = (df['future_price'] - df['close']) / df['close']
    
    df['label'] = 0 # Neutro
    df.loc[df['price_change'] > profit_target_pct, 'label'] = 1  # Comprar
    df.loc[df['price_change'] < -stop_loss_pct, 'label'] = -1 # Vender
    
    return df.dropna()

# --- FUNÇÃO 2: Adicionar Todas as Features ---
def add_all_features(df):
    df['sma_20'] = df['close'].rolling(window=20).mean()
    df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi_14'] = 100 - (100 / (1 + rs))
    df['bollinger_upper'] = df['sma_20'] + (df['close'].rolling(window=20).std() * 2)
    df['bollinger_lower'] = df['sma_20'] - (df['close'].rolling(window=20).std() * 2)
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema_12 - ema_26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['daily_return'] = df['close'].pct_change()
    df['day_of_week'] = df.index.dayofweek.astype(float)
    df['day_of_month'] = df.index.day.astype(float)
    df['week_of_year'] = df.index.isocalendar().week.astype(float)
    df['month'] = df.index.month.astype(float)
    return df.dropna()

# --- FUNÇÃO 3: Criar Sequências para a LSTM ---
def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

print("Funções do pipeline de otimização definidas com sucesso.")


Funções do pipeline de otimização definidas com sucesso.


In [3]:
# CÉLULA 2: O Loop de Otimização (CÓDIGO COMPLETO)

# --- PARÂMETROS PARA TESTAR ---
params_to_test = {
    'profit_target': [0.05, 0.10],
    'stop_loss': [0.025, 0.05],
    'confidence_threshold': [0.75, 0.80, 0.90]
}
look_forward = 20
time_steps = 60

# --- Carregar os dados brutos uma vez ---
btc_df_raw = pd.read_feather('../user_data/data/binance/BTC_USDT-1d.feather')
btc_df_raw['date'] = pd.to_datetime(btc_df_raw['date'])
btc_df_raw.set_index('date', inplace=True)

# --- Loop principal ---
results = []

for pt in params_to_test['profit_target']:
    for sl in params_to_test['stop_loss']:
        print(f"\\n--- INICIANDO TESTE COM: Alvo={pt*100:.1f}%, Stop={sl*100:.1f}% ---")
        
        # 1. PREPARAÇÃO DOS DADOS
        df_labeled = create_labels(btc_df_raw.copy(), look_forward, pt, sl)
        df_featured = add_all_features(df_labeled)
        
        y = df_featured['label']
        X = df_featured.drop(columns=['label', 'open', 'high', 'low', 'close', 'future_price', 'price_change'])
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train.values, time_steps)
        X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test.values, time_steps)
        
        y_train_adj = y_train_seq + 1
        
        # 2. TREINO DO MODELO
        model = Sequential([
            LSTM(units=50, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
            Dropout(0.2),
            LSTM(units=50, return_sequences=False),
            Dropout(0.2),
            Dense(units=3, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        model.fit(X_train_seq, y_train_adj, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=0)
        
        # 3. BACKTESTING
        predictions_test = model.predict(X_test_seq)
        
        start_index = len(X_train) + time_steps
        test_prices = df_featured['close'].iloc[start_index:]
        backtest_df = pd.DataFrame({'close_price': test_prices, 'market_return': test_prices.pct_change()})
        
        for conf in params_to_test['confidence_threshold']:
            backtest_df['signal'] = 0
            backtest_df.loc[predictions_test[:, 2] > conf, 'signal'] = 1
            backtest_df.loc[predictions_test[:, 0] > conf, 'signal'] = -1
            
            backtest_df['strategy_return'] = backtest_df['market_return'] * backtest_df['signal'].shift(1)
            cumulative_return = (1 + backtest_df['strategy_return']).cumprod().iloc[-1]
            
            results.append({
                'profit_target': pt,
                'stop_loss': sl,
                'confidence': conf,
                'strategy_return': (cumulative_return - 1) * 100
            })
            print(f"  Resultado para Confiança={conf*100}%: {((cumulative_return - 1) * 100):.2f}%")

# --- Exibir os resultados finais ---
results_df = pd.DataFrame(results)
print("\\n--- TABELA DE RESULTADOS DA OTIMIZAÇÃO ---")
display(results_df.sort_values(by='strategy_return', ascending=False))


\n--- INICIANDO TESTE COM: Alvo=5.0%, Stop=2.5% ---


2025-09-20 15:41:18.068607: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-09-20 15:41:18.068926: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-20 15:41:18.068932: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-20 15:41:18.068982: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-20 15:41:18.069046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/marianobernini/miniconda/envs/botenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When u

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
  Resultado para Confiança=75.0%: -40.42%
  Resultado para Confiança=80.0%: -28.44%
  Resultado para Confiança=90.0%: 0.00%
\n--- INICIANDO TESTE COM: Alvo=5.0%, Stop=5.0% ---


/Users/marianobernini/miniconda/envs/botenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
  Resultado para Confiança=75.0%: -17.48%
  Resultado para Confiança=80.0%: 0.00%
  Resultado para Confiança=90.0%: 0.00%
\n--- INICIANDO TESTE COM: Alvo=10.0%, Stop=2.5% ---


/Users/marianobernini/miniconda/envs/botenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
  Resultado para Confiança=75.0%: -40.43%
  Resultado para Confiança=80.0%: -23.73%
  Resultado para Confiança=90.0%: 0.00%
\n--- INICIANDO TESTE COM: Alvo=10.0%, Stop=5.0% ---


/Users/marianobernini/miniconda/envs/botenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
  Resultado para Confiança=75.0%: -0.84%
  Resultado para Confiança=80.0%: 0.00%
  Resultado para Confiança=90.0%: 0.00%
\n--- TABELA DE RESULTADOS DA OTIMIZAÇÃO ---


,profit_target,stop_loss,confidence,strategy_return
2,0.05,0.025,0.90,0.000000
4,0.05,0.050,0.80,0.000000
5,0.05,0.050,0.90,0.000000
8,0.10,0.025,0.90,0.000000
10,0.10,0.050,0.80,0.000000
11,0.10,0.050,0.90,0.000000
9,0.10,0.050,0.75,-0.844846
3,0.05,0.050,0.75,-17.476562
7,0.10,0.025,0.80,-23.727363
1,0.05,0.025,0.80,-28.442600
